In [1]:
# !pip install pandas 
# !pip install pandasai pandasai-litellm

In [2]:
import pandas as pd
df = pd.read_csv("../data/shipments_1000.csv", parse_dates=["date"])
max_date = df['date'].max()
last_week = df[(df['date'] >= max_date - pd.Timedelta(days=6)) & (df['date'] <= max_date)]
# 1) Most delays last week
delays_last_week = last_week[last_week['delay_minutes'] > 0]
print(delays_last_week['route'].value_counts())

# 2) Total delayed shipments by reason
delayed = df[df['delay_minutes'] > 0]
print(delayed['delay_reason'].value_counts())

# 3) Warehouses avg delivery_time > 5
print(df.groupby('warehouse')['delivery_time'].mean().loc[lambda x: x > 5])

# 4) Average delay in October
print(df[df['date'].dt.month == 10]['delay_minutes'].mean())

# 5) Simple prediction: use last 28 days delay fraction
last_28 = df[(df['date'] >= max_date - pd.Timedelta(days=27)) & (df['date'] <= max_date)]
pred_rate = (last_28['delay_minutes'] > 0).mean()
print("Predicted delay rate next week (frac):", pred_rate)

route
Route A    4
Route C    4
Route D    2
Route B    2
Route E    1
Name: count, dtype: int64
delay_reason
Traffic             157
Driver Rest         145
Mechanical Issue    143
Weather             137
Minor Breakdown     133
Customs Delay       115
Name: count, dtype: int64
warehouse
WH1    5.527610
WH2    5.527119
WH3    5.464298
WH4    5.451840
Name: delivery_time, dtype: float64
49.935483870967744
Predicted delay rate next week (frac): 0.8214285714285714


In [3]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd 
import logging
import pandasai as pai
from pandasai import SmartDataframe
from pandasai.helpers.logger import Logger
from pandasai_litellm.litellm import LiteLLM

logger = Logger(save_logs=True, verbose=False)
llm = LiteLLM(
    model="openai/Qwen/Qwen3-14B-AWQ",
    base_url="http://localhost:8001/v1",
    api_key="EMPTY"
)
pai.config.set({
    "llm": llm,
    "logger": logger
})

df = pai.read_csv("../data/shipments_1000.csv")
time_ctx = f"<TIME_CONTEXT>Current date: {df.date.max()}</TIME_CONTEXT> "

def _create_prompt(question):
    return time_ctx + question + " /no_think"

In [4]:
response = df.chat(_create_prompt("Which route had the most delays last week"))
response

StringResponse(type='string', value='The route with the most delays last week was Route C with 4 delays.')

In [5]:
response = df.chat(_create_prompt("Show total delayed shipments by delay reason"))
response

DataFrameResponse(type='dataframe', value=       delay_reason  total_delayed_shipments
0           Traffic                      157
1       Driver Rest                      145
2  Mechanical Issue                      143
3           Weather                      137
4   Minor Breakdown                      133
5     Customs Delay                      115)

In [6]:
response = df.chat(_create_prompt("List warehouses with average delivery time above 5 days"))
response

DataFrameResponse(type='dataframe', value=  warehouse  avg_delivery_time
0       WH1           5.527610
1       WH2           5.527119
2       WH3           5.464298
3       WH4           5.451840)

In [7]:
response = df.chat(_create_prompt("What was the average delay in October?"))
response

NumberResponse(type='number', value=49.935483870967744)

In [8]:
response = df.chat(_create_prompt("Predict the delay rate for next week"))
response

NumberResponse(type='number', value=52.784)

In [9]:
questions = pd.read_csv("../data/shipment_questions_500.csv")
questions.sample(10)

,id,question,code_concise,code_verbose
88,89,Which route had the longest average delay in A...,"result = {'shape': df.shape, 'columns': list(d...","result = {'shape': df.shape, 'columns': list(d..."
268,269,Show the bottom 3 warehouses by average delive...,"result = {'shape': df.shape, 'columns': list(d...","result = {'shape': df.shape, 'columns': list(d..."
452,453,Which warehouse had the most weather-related d...,"result = {'shape': df.shape, 'columns': list(d...","result = {'shape': df.shape, 'columns': list(d..."
387,388,Which route had the most Driver Rest delays?,"result = {'shape': df.shape, 'columns': list(d...","result = {'shape': df.shape, 'columns': list(d..."
141,142,How many shipments occurred on Thursdays?,"result = {'shape': df.shape, 'columns': list(d...","result = {'shape': df.shape, 'columns': list(d..."
499,500,Which routes had no delays at all? (count only),"result = [r for r,g in df.groupby('route') if ...",groups = df.groupby('route')\n# routes with ze...
260,261,Which warehouse had the most shipments in August?,"result = {'shape': df.shape, 'columns': list(d...","result = {'shape': df.shape, 'columns': list(d..."
145,146,List shipments in January with delay_minutes >...,"result = {'shape': df.shape, 'columns': list(d...","result = {'shape': df.shape, 'columns': list(d..."
95,96,Which route had the most Minor Breakdown delays?,"result = {'shape': df.shape, 'columns': list(d...","result = {'shape': df.shape, 'columns': list(d..."
446,447,How many shipments took less than 3 days? (det...,result = (df['delivery_time']<3).sum(),# shipments with delivery_time less than 3\nre...


In [10]:
def execute_code(code):
    exec_locals = {"df": df, "pd": pd}
    exec(code, {}, exec_locals)
    return exec_locals.get("result", None)

questions["answer"] = questions["code_concise"].apply(execute_code)

In [11]:
questions

,id,question,code_concise,code_verbose,answer
0,1,How many total shipments are in the dataset?,result = len(df),df['date']=pd.to_datetime(df['date'])\n# total...,1000
1,2,What is the earliest shipment date recorded?,df['date']=pd.to_datetime(df['date']); result ...,df['date']=pd.to_datetime(df['date'])\n# earli...,2024-01-01 00:00:00
2,3,What is the most recent shipment date?,df['date']=pd.to_datetime(df['date']); result ...,df['date']=pd.to_datetime(df['date'])\n# most ...,2024-12-30 00:00:00
3,4,How many unique routes are there?,result = df['route'].nunique(),# number of unique routes\nresult = df['route'...,5
4,5,How many warehouses are involved?,result = df['warehouse'].nunique(),# number of unique warehouses\nresult = df['wa...,4
...,...,...,...,...,...
495,496,How many shipments occurred per month? (detailed),df['date']=pd.to_datetime(df['date']); result ...,df['date']=pd.to_datetime(df['date'])\n# shipm...,date 1 92 2 87 3 93 4 90 5 ...
496,497,Which warehouse had the least total delay minu...,result = df['delay_minutes'].sum(),# total delay minutes across dataset\nresult =...,52784
497,498,Which week had the most deliveries? (detailed),df['date']=pd.to_datetime(df['date']); result ...,df['date']=pd.to_datetime(df['date'])\n# week ...,1
498,499,How many shipments had zero delay? (count only),result = (df['delay_minutes']==0).sum(),# count shipments with zero delay\nresult = df...,170


In [12]:
from tqdm import tqdm

predicts = []
for i, question in tqdm(questions.iterrows()):
    try:
        predict = df.chat(_create_prompt(question["question"]))
    except:
        predict = ""
    predicts.append(predict)
    
questions["predict"] = predicts

0it [00:00, ?it/s]

500it [27:22,  3.28s/it]


In [14]:
questions

,id,question,code_concise,code_verbose,answer,predict
0,1,How many total shipments are in the dataset?,result = len(df),df['date']=pd.to_datetime(df['date'])\n# total...,1000,1000
1,2,What is the earliest shipment date recorded?,df['date']=pd.to_datetime(df['date']); result ...,df['date']=pd.to_datetime(df['date'])\n# earli...,2024-01-01 00:00:00,The earliest shipment date recorded is 2024-01...
2,3,What is the most recent shipment date?,df['date']=pd.to_datetime(df['date']); result ...,df['date']=pd.to_datetime(df['date'])\n# most ...,2024-12-30 00:00:00,The most recent shipment date is 2024-12-30 00...
3,4,How many unique routes are there?,result = df['route'].nunique(),# number of unique routes\nresult = df['route'...,5,5
4,5,How many warehouses are involved?,result = df['warehouse'].nunique(),# number of unique warehouses\nresult = df['wa...,4,4
...,...,...,...,...,...,...
495,496,How many shipments occurred per month? (detailed),df['date']=pd.to_datetime(df['date']); result ...,df['date']=pd.to_datetime(df['date'])\n# shipm...,date 1 92 2 87 3 93 4 90 5 ...,month shipments_count\n0 2024-01 ...
496,497,Which warehouse had the least total delay minu...,result = df['delay_minutes'].sum(),# total delay minutes across dataset\nresult =...,52784,The warehouse with the least total delay minut...
497,498,Which week had the most deliveries? (detailed),df['date']=pd.to_datetime(df['date']); result ...,df['date']=pd.to_datetime(df['date'])\n# week ...,1,week deliveries\n0 2024-04 21
498,499,How many shipments had zero delay? (count only),result = (df['delay_minutes']==0).sum(),# count shipments with zero delay\nresult = df...,170,170


In [13]:
questions.to_csv("../data/processed/predicts.csv")

In [ ]:
<TIME_CONTEXT>
Current date: {df.date.max()}
</TIME_CONTEXT>    
    

In [3]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8001/v1",
    api_key="EMPTY"
)

def ask_llm(question: str):
    prompt = f"""

You are a data analyst. Convert this question into Python pandas code that answers it using the DataFrame `df`.
Return only excutable code.

Question: {question}
"""
    resp = client.chat.completions.create(
        model="Qwen/Qwen3-14B-AWQ", # ,
        messages=[
            {
                "role": "system",
                "content": ""
            },
            {"role": "user", "content": prompt}
        ]
    )
    code = resp.choices[0].message.content
    print(f"Generated code:\n {code}")
    return code

In [4]:
code = ask_llm("Which route had the most delays last week")

Generated code:
 <think>
Okay, the user is asking which route had the most delays last week. Let me think about how to approach this with pandas.

First, I need to make sure I understand the DataFrame structure. The question mentions 'route' and 'delays', so there should be columns for route (maybe 'route_id' or 'route_number') and some measure of delays. But since the user hasn't specified the exact column names, I'll have to make some assumptions.

Assuming the DataFrame has a 'route' column that identifies each route, and a 'delay' column indicating the delay time or a binary indicator (like 1 for delayed, 0 otherwise). But since the question is about the number of delays, maybe it's the count of delayed flights per route. Alternatively, there might be a 'delay_minutes' column, and we need to sum those.

Wait, the question says "most delays", which could mean either the highest number of delayed instances or the total delay time. But without specific info, I'll assume that each row 

In [ ]:
exec_locals = {"df": df}
exec(code, {}, exec_locals)
exec_locals.get("result", None)